# Paso 8. Ecuación Bidimensional de Burguers'

### Forma vectorial

$$
\frac{D \vec{v}}{D t} = \nu \nabla^2 \vec{v}
$$

### Forma expandida en $\mathbb{R}^2$
$$
\frac{\partial u}{\partial t} + u\frac{\partial u}{\partial x} + v\frac{\partial u}{\partial y} = \nu \Bigg( \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} \Bigg)
$$

$$
\frac{\partial v}{\partial t} + u\frac{\partial v}{\partial x} + v\frac{\partial v}{\partial y} = \nu \Bigg( \frac{\partial^2 v}{\partial x^2} + \frac{\partial^2 v}{\partial y^2} \Bigg)
$$

### Discretización
Usando diferencias finitas

#### Eje X
$$
\frac{u(i,j)_{n+1} - u(i,j)_{n}}{\Delta t} + u(i,j)_{n}\frac{u(i,j)_n-u(i-1,j)_n}{\Delta x} + v(i,j)_{n}\frac{u(i,j)_n-u(i,j-1)_n}{\Delta y} = \nu \Bigg( \frac{u(i-1,j)_n - 2u(i,j)_n + u(i+1,j)_n}{\Delta x^2} + \frac{u(i,j-1)_n - 2u(i,j)_n + u(i,j+1)_n}{\Delta y^2} \Bigg)
$$

#### Eje Y
$$
\frac{v(i,j)_{n+1} - v(i,j)_{n}}{\Delta t} + u(i,j)_{n}\frac{v(i,j)_n-v(i-1,j)_n}{\Delta x} + v(i,j)_{n}\frac{v(i,j)_n-v(i,j-1)_n}{\Delta y} = \nu \Bigg( \frac{v(i-1,j)_n - 2v(i,j)_n + v(i+1,j)_n}{\Delta x^2} + \frac{v(i,j-1)_n - 2v(i,j)_n + v(i,j+1)_n}{\Delta y^2} \Bigg)
$$

#### Resolviendo para $u(i,j)_{n+1}$ en el eje X
$$
u(i,j)_{n+1} = u(i,j)_{n} 
- \frac{ u(i,j)_{n}\Delta t}{\Delta x}\Big[ u(i,j)_n-u(i-1,j)_n \Big]
- \frac{ v(i,j)_{n}\Delta t}{\Delta y}\Big[ u(i,j)_n-u(i,j-1)_n \Big] 
+ \Delta t\ \nu \Bigg( \frac{u(i-1,j)_n - 2u(i,j)_n + u(i+1,j)_n}{\Delta x^2} + \frac{u(i,j-1)_n - 2u(i,j)_n + u(i,j+1)_n}{\Delta y^2} \Bigg)
$$

#### Resolviendo para $v(i,j)_{n+1}$ en el eje Y
$$
v(i,j)_{n+1} = v(i,j)_{n} 
- \frac{ u(i,j)_{n}\Delta t}{\Delta x}\Big[ v(i,j)_n-v(i-1,j)_n \Big]
- \frac{ v(i,j)_{n}\Delta t}{\Delta y}\Big[ v(i,j)_n-v(i,j-1)_n \Big] 
+ \Delta t\ \nu \Bigg( \frac{v(i-1,j)_n - 2v(i,j)_n + v(i+1,j)_n}{\Delta x^2} + \frac{v(i,j-1)_n - 2v(i,j)_n + v(i,j+1)_n}{\Delta y^2} \Bigg)
$$

In [5]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D

def update(i, profile):
    #store the actual state of the arrays
    u_n = u;
    v_n = v
    
    #update arrays, just for readability create dummy variables
    un_i_j   = u_n[1:,1:];
    un_im1_j = np.roll(u,  1, axis=1)[1:, 1:];
    un_ip1_j = np.roll(u, -1, axis=1)[1:, 1:];
    un_i_jm1 = np.roll(u,  1, axis=0)[1:, 1:];
    un_i_jp1 = np.roll(u, -1, axis=0)[1:, 1:];
    
    vn_i_j   = v_n[1:,1:];
    vn_im1_j = np.roll(v,  1, axis=1)[1:, 1:];
    vn_ip1_j = np.roll(v, -1, axis=1)[1:, 1:];
    vn_i_jm1 = np.roll(v,  1, axis=0)[1:, 1:];
    vn_i_jp1 = np.roll(v, -1, axis=0)[1:, 1:];
    
    u[1:,1:] = un_i_j - (un_i_j*(dt/dx)) * (un_i_j - un_im1_j) - (vn_i_j*(dt/dy)) * (un_i_j - un_i_jm1) + \
                nu*(dt/dx**2)*( un_im1_j - 2*un_i_j + un_ip1_j ) + nu*(dt/dy**2)*( un_i_jm1 - 2*un_i_j + un_i_jp1 );
        
    v[1:,1:] = vn_i_j - (un_i_j*(dt/dx)) * (vn_i_j - vn_im1_j) - (vn_i_j*(dt/dy)) * (vn_i_j - vn_i_jm1) + \
                nu*(dt/dx**2)*( vn_im1_j - 2*vn_i_j + vn_ip1_j ) + nu*(dt/dy**2)*( vn_i_jm1 - 2*vn_i_j + vn_i_jp1 );
    
    ax.clear();
    ax.set_zlim(0.0, 0.16);
    ax.grid(False);

    profile = ax.plot_surface(x, y, u);
    
    #Animate Point of view.
    #ax.view_init(elev=15., azim=i)
    return profile,


# How much we want to advance in time?
# at 30 fps if we want simulate 10 seconds => 30*10
t_seconds = 30 * 40;

# The time step size 
dt = 0.09;

# viscocity
nu = 0.01

partitions = 101;

data_x = np.linspace(-3.0, 8.0, num=partitions, retstep=True);
x  = data_x[0]
dx = data_x[1];

data_y = np.linspace(-3.0, 8.0, num=partitions, retstep=True);
y  = data_x[0]
dy = data_x[1];

x, y = np.meshgrid(x, y)
#parameters for 2D normal distribution
sigma = np.ones(2) * 1;
miu   = np.ones(2) * 0.01;
rho = 0.0;

A = 2*np.pi*sigma[0]*sigma[1]*np.sqrt(1-rho**2);
B = 2*(1-rho**2);
C = (x-miu[0])**2 / sigma[0]**2;
D = (y-miu[1])**2 / sigma[1]**2;
E = (2*rho*(x-miu[0])*(y-miu[1])) / (miu[0]*miu[1]);

u = 1 / A * np.exp( -1/B * ( C + D - E ) );
v = 1 / A * np.exp( -1/B * ( C + D - E ) );

# Create a figure and a 3D Axes
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d');
ax.set_zlim(0, 0.16);
ax.grid(False);

profile = ax.plot_surface(x, y, u);

# Animate
anim = animation.FuncAnimation(fig, update, fargs=(profile,), frames=t_seconds, blit=False)
anim.save('2D_Burguers_01.mp4', fps=30, writer="ffmpeg", codec="libx264");
plt.close(fig);

In [6]:
# You must have configured ffmpeg in your machine to run the code below
from IPython.display import HTML
video = open("2D_Burguers_01.mp4", "rb").read()
video_encoded = video.encode("base64")
video_tag = '<video controls alt="test" src="data:video/x-m4v;base64,{0}">'.format(video_encoded)
HTML(video_tag)